In [1]:
%load_ext cython

In [2]:
import pythran
%load_ext pythran.magic

In [3]:
import numpy as np

# hausdorff

## cython version

In [4]:
%%cython
#
# Copyright (C)  Tyler Reddy, Richard Gowers, and Max Linke, 2016
#
# Distributed under the same BSD license as Scipy.
#

from __future__ import absolute_import

import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport sqrt


@cython.boundscheck(False)
def directed_hausdorff_cython(double[:,::1] ar1, double[:,::1] ar2, seed=0):

    cdef double cmax, cmin, d
    cdef bint no_break_occurred
    cdef int N1 = ar1.shape[0]
    cdef int N2 = ar2.shape[0]
    cdef int data_dims = ar1.shape[1]
    cdef unsigned int i, j, k
    cdef unsigned int i_store = 0, j_store = 0, i_ret = 0, j_ret = 0
    cdef long[:] resort1, resort2

    # shuffling the points in each array generally increases the likelihood of
    # an advantageous break in the inner search loop and never decreases the
    # performance of the algorithm
    rng = np.random.RandomState(seed)
    resort1 = np.arange(N1)
    resort2 = np.arange(N2)
    rng.shuffle(resort1)
    rng.shuffle(resort2)
    ar1 = np.asarray(ar1)[resort1]
    ar2 = np.asarray(ar2)[resort2]

    cmax = 0
    for i in range(N1):
        no_break_occurred = True
        cmin = np.inf
        for j in range(N2):
            d = 0
	    # faster performance with square of distance
	    # avoid sqrt until very end
            for k in range(data_dims):
                d += (ar1[i, k] - ar2[j, k])**2
            if d < cmax: # break out of `for j` loop
                no_break_occurred = False
                break

            if d < cmin: # always true on first iteration of for-j loop
                cmin = d
                i_store = i
                j_store = j

        # always true on first iteration of for-j loop, after that only
        # if d >= cmax
        if cmin != np.inf and cmin > cmax and no_break_occurred == True:
            cmax = cmin
            i_ret = i_store
            j_ret = j_store

    return (sqrt(cmax), resort1[i_ret], resort2[j_ret])


## pythran version

In [5]:
%%pythran
# Author: Eric Larson
# 2014
import numpy as np
import numpy.random as random

#pythran export directed_hausdorff_pythran(float64[:,:], float64[:,:], int)

def directed_hausdorff_pythran(ar1, ar2, seed=0):

    N1, data_dims = ar1.shape
    N2 = ar2.shape[0]
    i_store = j_store = i_ret = j_ret = 0

    # shuffling the points in each array generally increases the likelihood of
    # an advantageous break in the inner search loop and never decreases the
    # performance of the algorithm
    random.seed(seed)
    resort1 = np.arange(N1)
    resort2 = np.arange(N2)
    random.shuffle(resort1)
    random.shuffle(resort2)
    ar1 = np.asarray(ar1)[resort1]
    ar2 = np.asarray(ar2)[resort2]

    cmax = 0
    for i in range(N1):
        cmin = np.inf
        for j in range(N2):
            d = np.sum((ar1[i] - ar2[j]) ** 2)
            # faster performance with square of distance
            # avoid sqrt until very end
            if d < cmax: # break out of `for j` loop
                break

            if d < cmin: # always true on first iteration of for-j loop
                cmin = d
                i_store = i
                j_store = j
        else:

            # always true on first iteration of for-j loop, after that only
            # if d >= cmax
            if cmin != np.inf and cmin > cmax:
                cmax = cmin
                i_ret = i_store
                j_ret = j_store

    return np.sqrt(cmax), resort1[i_ret], resort2[j_ret]


## bench

In [6]:
n = 200
args = np.arange((n * n), dtype=float).reshape(n,-1), np.ones((n,n)) * 3., 0

In [7]:
%timeit directed_hausdorff_cython(*args)

851 µs ± 24.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%timeit directed_hausdorff_pythran(*args)

542 µs ± 43.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
